In [1]:
# basics
import numpy as np
import pandas as pd
import datetime
import re
import os, glob
import pickle
import textwrap

# statistics
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import KFold
from datetime import date, timedelta
import tqdm


# plotting
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [15,12]
from matplotlib.ticker import MultipleLocator

In [3]:
data = pd.read_csv('Data/statsmodels.csv', sep=';', index_col='Date', parse_dates=['Date']).drop(['Unnamed: 0'], axis=1)
display(data.tail())

main = pd.read_csv('Data/statsmodels rel.csv', sep=';', index_col='Date', parse_dates=['Date']).drop(['Unnamed: 0'], axis=1)
display(main.tail())

deep = pd.read_csv('Data/statsmodels deep rel.csv', sep=';', index_col='Date', parse_dates=['Date']).drop(['Unnamed: 0'], axis=1)
display(deep.tail())

,No_cases,Rolling_R,Workplace,Events,Daycare,Higher Education,Secondary Education,Primary Education,Restaurants/Cafes,Transport,...,Masks,MassGather50,MassGatherAll,Shops,Outdoor,Religion,Gatherings,Stayhome,Telework,Prikken
Date,,,,,,,,,,,,,,,,,,,,,
2021-05-02,5753,0.907408,2.0,2.0,0.0,2.0,1.0,0.0,2.0,0.0,...,2.0,2.0,2.0,1.0,0.0,0.0,2.0,3.0,2.0,5651041.0
2021-05-03,5716,0.893238,2.0,2.0,0.0,2.0,1.0,0.0,2.0,0.0,...,2.0,2.0,2.0,1.0,0.0,0.0,2.0,3.0,2.0,5757731.0
2021-05-04,5336,0.880369,2.0,2.0,0.0,2.0,1.0,0.0,2.0,0.0,...,2.0,2.0,2.0,1.0,0.0,0.0,2.0,3.0,2.0,5860446.0
2021-05-05,5936,0.869616,2.0,2.0,0.0,2.0,1.0,0.0,2.0,0.0,...,2.0,2.0,2.0,1.0,0.0,0.0,2.0,3.0,2.0,5980676.0
2021-05-06,5365,0.860512,2.0,2.0,0.0,2.0,1.0,0.0,2.0,0.0,...,2.0,2.0,2.0,1.0,0.0,0.0,2.0,3.0,2.0,6101271.0


,No_cases,Workplace,Events,Daycare,Higher Education,Secondary Education,Primary Education,Restaurants/Cafes,Transport,Entertainment,...,MassGather50,MassGatherAll,Shops,Outdoor,Religion,Gatherings,Stayhome,Telework,Prikken,Rolling_R
Date,,,,,,,,,,,,,,,,,,,,,
2021-05-02,0.457531,1.0,1.0,0.0,1.0,0.5,0.0,1.0,0.0,1.0,...,1.0,1.0,0.5,0.0,0.0,1.0,0.75,1.0,0.926207,0.907408
2021-05-03,0.454589,1.0,1.0,0.0,1.0,0.5,0.0,1.0,0.0,1.0,...,1.0,1.0,0.5,0.0,0.0,1.0,0.75,1.0,0.943694,0.893238
2021-05-04,0.424368,1.0,1.0,0.0,1.0,0.5,0.0,1.0,0.0,1.0,...,1.0,1.0,0.5,0.0,0.0,1.0,0.75,1.0,0.960529,0.880369
2021-05-05,0.472085,1.0,1.0,0.0,1.0,0.5,0.0,1.0,0.0,1.0,...,1.0,1.0,0.5,0.0,0.0,1.0,0.75,1.0,0.980234,0.869616
2021-05-06,0.426674,1.0,1.0,0.0,1.0,0.5,0.0,1.0,0.0,1.0,...,1.0,1.0,0.5,0.0,0.0,1.0,0.75,1.0,1.000000,0.860512


,No_cases,Workplace,Events,Daycare,Higher Education,Secondary Education,Primary Education,Restaurants/Cafes,Transport,Entertainment,...,Outdoor,Religion,Gatherings,Stayhome,Telework,Prikken,Rolling_R,perc_randstad,perc_under_30,perc_M
Date,,,,,,,,,,,,,,,,,,,,,
2021-05-02,0.457531,1.0,1.0,0.0,1.0,0.5,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.75,1.0,0.926207,0.907408,0.425865,0.404311,0.504085
2021-05-03,0.454589,1.0,1.0,0.0,1.0,0.5,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.75,1.0,0.943694,0.893238,0.441393,0.414626,0.506298
2021-05-04,0.424368,1.0,1.0,0.0,1.0,0.5,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.75,1.0,0.960529,0.880369,0.432534,0.434970,0.513306
2021-05-05,0.472085,1.0,1.0,0.0,1.0,0.5,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.75,1.0,0.980234,0.869616,0.443902,0.433120,0.517352
2021-05-06,0.426674,1.0,1.0,0.0,1.0,0.5,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.75,1.0,1.000000,0.860512,0.436906,0.448462,0.520969


In [4]:
def shift_outcome(X,y,s,shift_date="y"):

    # If shift_date="y", then the date of y is shifted s days into the past. If shift_date="X", then the date of X is shifted s days into the future.

        if shift_date=="y":
            
            y_shifted = y.reset_index().copy()
            y_shifted["Date"] = y_shifted[["Date"]].applymap(lambda x: x.date()-timedelta(s))
            y_shifted = y_shifted.set_index(["Date"])
            y_shifted.drop(y_shifted.loc[np.isnan(y_shifted.values)].index,inplace=True)
            data2 = X.join(y_shifted,how="inner")
            X = data2[X.columns]
            y_shifted = data2[y.name]
            
            return X,y_shifted
        
        if shift_date=="X":
        
            X_shifted = X.reset_index().copy()
            X_shifted["Date"] = X_shifted[["Date"]].applymap(lambda x: x.date()+timedelta(s))
            X_shifted = X_shifted.set_index(["Date"])
            y.drop(y.loc[np.isnan(y.values)].index,inplace=True)
            data = X_shifted.join(y,how="inner")
            X_shifted = data[X.columns]
            y = data[y.name]
            
            return X_shifted,y

# GENDER

In [36]:
%%time
shiftdays = range(0, 10) 
alphalist = np.power(10,np.linspace(-4,-1,num=17)) 
crossvalcount = 10

# shiftdays = [1, 2, 6]
# alphalist = [0.01, 0.5, 1e-3]


X = deep.drop(['No_cases', 'Rolling_R', 'perc_randstad', 'perc_under_30', 'perc_M'], axis=1)
Y = deep['perc_M']

curResDF = pd.DataFrame()
n = 0
totaln = len(shiftdays) * len(alphalist)
for s in shiftdays:
    
    X_used, Y_used = shift_outcome(X,Y,s,shift_date="y")
    
    X_used.columns = [col.replace(' ', '_').replace('/', '_') for col in X_used.columns.values]
    combined = X_used.copy()
    combined['perc_M'] = Y_used
    
    formula = 'perc_M ~ ' + ' + '.join(X_used.columns)
    
    for a in alphalist:
        print(f'Huidige progressie: {(n / totaln)*100:.2f}% \n shiftday: {s}, alpha: {a}')
        kf = KFold(n_splits=crossvalcount, shuffle=True)

        for train_ind, test_ind in kf.split(X_used):
            #print(train_ind, test_ind)
            train = combined.iloc[train_ind]
            test = combined.iloc[test_ind]
#           
            trainmodel    = smf.ols(formula = formula, data = train)
            testmodel     = smf.ols(formula = formula, data = test)
        
            trainresults  = trainmodel.fit(alpha = a, L1_wt = 1)

            test_params   = []
            for paramname in testmodel.exog_names:
                if paramname in trainresults.params.keys():
                    test_params.append(trainresults.params[paramname])
                else:
                    test_params.append(0)

            obs_train     = np.array(trainmodel.endog)
            obs_test      = np.array(testmodel.endog)
            pred_train    = trainmodel.predict(trainresults.params)
            pred_test     = testmodel.predict(test_params)
            
            result_dict                          = {'shiftdays': s, 'alpha': a}
            result_dict['Test Sample Size']      = len(test)
            result_dict['Training Sample Size']  = len(train)
            result_dict['Loglike Training']      = trainmodel.loglike(trainresults.params)
            result_dict['Loglike Test']          = testmodel.loglike(np.array(test_params))
            result_dict['RSS Training']          = np.sum((obs_train - pred_train)**2)
            result_dict['RSS Test']              = np.sum((obs_test - pred_test)**2)
            result_dict['NVar Training']         = np.sum((obs_train - np.mean(obs_train))**2)
            result_dict['NVar Test']             = np.sum((obs_test - np.mean(obs_test))**2)
            result_dict['R2 Training']           = 1 - result_dict['RSS Training']/result_dict['NVar Training']
            result_dict['R2 Test']               = 1 - result_dict['RSS Test']/result_dict['NVar Test']
            result_dict['F value']               = trainresults.fvalue
            result_dict['P value']               = trainresults.f_pvalue

            result_dict.update({k:v for k,v in trainresults.params.items()})
            new_DF = pd.DataFrame({k:np.array([v]) for k,v in result_dict.items()})
            curResDF = curResDF.append(new_DF, ignore_index=False)
        n += 1

Huidige progressie: 0.00% 
 shiftday: 0, alpha: 0.0001
Huidige progressie: 0.59% 
 shiftday: 0, alpha: 0.0001539926526059492
Huidige progressie: 1.18% 
 shiftday: 0, alpha: 0.00023713737056616554
Huidige progressie: 1.76% 
 shiftday: 0, alpha: 0.0003651741272548377
Huidige progressie: 2.35% 
 shiftday: 0, alpha: 0.0005623413251903491
Huidige progressie: 2.94% 
 shiftday: 0, alpha: 0.0008659643233600654
Huidige progressie: 3.53% 
 shiftday: 0, alpha: 0.001333521432163324
Huidige progressie: 4.12% 
 shiftday: 0, alpha: 0.002053525026457146
Huidige progressie: 4.71% 
 shiftday: 0, alpha: 0.0031622776601683794
Huidige progressie: 5.29% 
 shiftday: 0, alpha: 0.004869675251658631
Huidige progressie: 5.88% 
 shiftday: 0, alpha: 0.007498942093324558
Huidige progressie: 6.47% 
 shiftday: 0, alpha: 0.011547819846894581
Huidige progressie: 7.06% 
 shiftday: 0, alpha: 0.01778279410038923
Huidige progressie: 7.65% 
 shiftday: 0, alpha: 0.027384196342643614
Huidige progressie: 8.24% 
 shiftday: 0, a

Huidige progressie: 71.18% 
 shiftday: 7, alpha: 0.00023713737056616554
Huidige progressie: 71.76% 
 shiftday: 7, alpha: 0.0003651741272548377
Huidige progressie: 72.35% 
 shiftday: 7, alpha: 0.0005623413251903491
Huidige progressie: 72.94% 
 shiftday: 7, alpha: 0.0008659643233600654
Huidige progressie: 73.53% 
 shiftday: 7, alpha: 0.001333521432163324
Huidige progressie: 74.12% 
 shiftday: 7, alpha: 0.002053525026457146
Huidige progressie: 74.71% 
 shiftday: 7, alpha: 0.0031622776601683794
Huidige progressie: 75.29% 
 shiftday: 7, alpha: 0.004869675251658631
Huidige progressie: 75.88% 
 shiftday: 7, alpha: 0.007498942093324558
Huidige progressie: 76.47% 
 shiftday: 7, alpha: 0.011547819846894581
Huidige progressie: 77.06% 
 shiftday: 7, alpha: 0.01778279410038923
Huidige progressie: 77.65% 
 shiftday: 7, alpha: 0.027384196342643614
Huidige progressie: 78.24% 
 shiftday: 7, alpha: 0.042169650342858224
Huidige progressie: 78.82% 
 shiftday: 7, alpha: 0.06493816315762113
Huidige progress

In [37]:
CVresults = curResDF.copy()
CVresults['alpha'] = CVresults['alpha'].map('{:.6e}'.format)
CVresults =  CVresults.groupby(['shiftdays','alpha'], as_index = False).agg(
            { 'Loglike Test':['mean','std'],
              'Loglike Training':['mean','std'],
              'R2 Test': ['mean','std'],
              'R2 Training': ['mean','std'],
              'RSS Training' : ['sum'],
              'RSS Test': ['sum'],
              'NVar Training':['sum'],
              'NVar Test':['sum'],
              'Test Sample Size':['sum'],
              'Training Sample Size':['sum']
            })
CVresults.columns = [ 'shiftdays','alpha',
                      'Loglike Test Avgd','Loglike Test Avgd Std',
                      'Loglike Training Avgd','Loglike Training Avgd Std',
                      'R2 Test Avgd','R2 Test Avgd Std',
                      'R2 Training Avgd','R2 Training Avgd Std',
                      'RSS Training Sum',
                      'RSS Test Sum',
                      'NVar Training Sum',
                      'NVar Test Sum',
                      'Test Sample Size',
                      'Training Sample Size'
                    ]
CVresults['RSS per datapoint Training'] = CVresults['RSS Training Sum']/CVresults['Training Sample Size']
CVresults['RSS per datapoint Test']     = CVresults['RSS Test Sum']/CVresults['Test Sample Size']

CVresults['R2 Training Weighted']       = 1 - CVresults['RSS Training Sum']/CVresults['NVar Training Sum']
CVresults['R2 Test Weighted']           = 1 - CVresults['RSS Test Sum']/CVresults['NVar Test Sum']

CVresults['alpha']                      = CVresults['alpha'].astype(np.float64) # return to numbers

# manually select best
CVresults.sort_values(by='R2 Test Weighted', ascending=False)

,shiftdays,alpha,Loglike Test Avgd,Loglike Test Avgd Std,Loglike Training Avgd,Loglike Training Avgd Std,R2 Test Avgd,R2 Test Avgd Std,R2 Training Avgd,R2 Training Avgd Std,RSS Training Sum,RSS Test Sum,NVar Training Sum,NVar Test Sum,Test Sample Size,Training Sample Size,RSS per datapoint Training,RSS per datapoint Test,R2 Training Weighted,R2 Test Weighted
70,4,0.011548,100.111960,8.160936,914.088286,7.402864,0.825622,0.091782,0.860512,0.005485,2.094776,0.261794,15.025592,1.654295,433,3897,0.000538,0.000605,0.860586,0.841749
69,4,0.000100,100.502999,7.869207,914.153698,8.226229,0.829378,0.081955,0.860503,0.006104,2.094764,0.261203,15.024733,1.646839,433,3897,0.000538,0.000603,0.860579,0.841392
79,4,0.042170,99.567233,5.596302,914.111730,6.344096,0.828163,0.069036,0.860515,0.005580,2.094245,0.262495,15.025482,1.653342,433,3897,0.000537,0.000606,0.860620,0.841234
75,4,0.000237,99.698428,6.562852,914.071026,6.032127,0.827168,0.073831,0.860513,0.004799,2.094432,0.261672,15.024621,1.645673,433,3897,0.000537,0.000604,0.860600,0.840994
83,4,0.007499,100.348846,8.358892,914.085955,8.146227,0.815971,0.099871,0.860448,0.005336,2.094993,0.260761,15.023456,1.635173,433,3897,0.000538,0.000602,0.860552,0.840530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,9,0.027384,95.288373,8.807354,872.934133,10.542983,0.729548,0.238236,0.836649,0.011245,2.429004,0.318331,14.897809,1.602521,428,3852,0.000631,0.000744,0.836956,0.801356
166,9,0.000562,94.247366,7.358655,873.024753,8.811523,0.785539,0.105789,0.836922,0.010312,2.426883,0.325375,14.901454,1.635059,428,3852,0.000630,0.000760,0.837138,0.801001
157,9,0.000154,95.480526,10.315106,873.004460,9.109452,0.775671,0.124220,0.836767,0.009744,2.427366,0.320024,14.897697,1.600952,428,3852,0.000630,0.000748,0.837064,0.800104
165,9,0.004870,94.069715,8.782820,873.051679,8.195599,0.669546,0.276735,0.836830,0.008893,2.426060,0.329351,14.895393,1.580464,428,3852,0.000630,0.000770,0.837127,0.791611


In [70]:
shiftdays = 4
alpha = 0.011548
crossvalcount = 10

DF = pd.DataFrame()

X = deep.drop(['No_cases', 'Rolling_R', 'perc_randstad', 'perc_under_30', 'perc_M'], axis=1)
Y = deep['perc_M']

X_used, Y_used = shift_outcome(X,Y,shiftdays,shift_date="y")
X_used.columns = [col.replace(' ', '_').replace('/', '_') for col in X_used.columns.values]
combined = X_used.copy()
combined['perc_M'] = Y_used
formula = 'perc_M ~ ' + ' + '.join(X_used.columns)
print(formula)
        
kf = KFold(n_splits=crossvalcount, shuffle=True)

for train_ind, test_ind in kf.split(X_used):
#   print(train_ind, test_ind)
    train = combined.iloc[train_ind]
    test = combined.iloc[test_ind]
       
    trainmodel    = smf.ols(formula = formula, data = train)
    testmodel     = smf.ols(formula = formula, data = test)

    trainresults  = trainmodel.fit(alpha = alpha, L1_wt = 1)#lasso
    print(trainresults.summary())

    test_params   = []
    for paramname in testmodel.exog_names:
        if paramname in trainresults.params.keys():
            test_params.append(trainresults.params[paramname])
        else:
            test_params.append(0)

    obs_train     = np.array(trainmodel.endog)
    obs_test      = np.array(testmodel.endog)
    pred_train    = trainmodel.predict(trainresults.params)
    pred_test     = testmodel.predict(test_params)

    result_dict                          = {'shiftdays': shiftdays, 'alpha': alpha}
    result_dict['Test Sample Size']      = len(test)
    result_dict['Training Sample Size']  = len(train)
    result_dict['Loglike Training']      = trainmodel.loglike(trainresults.params)
    result_dict['Loglike Test']          = testmodel.loglike(np.array(test_params))
    result_dict['RSS Training']          = np.sum((obs_train - pred_train)**2)
    result_dict['RSS Test']              = np.sum((obs_test - pred_test)**2)
    result_dict['NVar Training']         = np.sum((obs_train - np.mean(obs_train))**2)
    result_dict['NVar Test']             = np.sum((obs_test - np.mean(obs_test))**2)
    result_dict['R2 Test']               = 1 - result_dict['RSS Test']/result_dict['NVar Test']
    result_dict['F value']               = trainresults.fvalue
    result_dict['P value']               = trainresults.f_pvalue
    result_dict['R2 Training']           = trainresults.rsquared
    

    result_dict.update({k:v for k,v in trainresults.params.items()})
    new_DF = pd.DataFrame({k:np.array([v]) for k,v in result_dict.items()})
    DF = DF.append(new_DF, ignore_index=False)
    
# formula = 'Rolling_R ~ ' + ' + '.join(X_used.columns)  
# finalmodel = smf.ols(data = combined, formula = formula)
# finalresults = finalmodel.fit_regularized(alpha=alpha, L1_wt = 1) #lasso
DF

perc_M ~ Workplace + Events + Daycare + Higher_Education + Secondary_Education + Primary_Education + Restaurants_Cafes + Transport + Entertainment + Sports + Indoor + Masks + MassGather50 + MassGatherAll + Shops + Outdoor + Religion + Gatherings + Stayhome + Telework + Prikken
                            OLS Regression Results                            
Dep. Variable:                 perc_M   R-squared:                       0.867
Model:                            OLS   Adj. R-squared:                  0.860
Method:                 Least Squares   F-statistic:                     126.2
Date:                Fri, 04 Jun 2021   Prob (F-statistic):          2.23e-148
Time:                        20:32:00   Log-Likelihood:                 915.01
No. Observations:                 389   AIC:                            -1790.
Df Residuals:                     369   BIC:                            -1711.
Df Model:                          19                                         
Covariance 

                            OLS Regression Results                            
Dep. Variable:                 perc_M   R-squared:                       0.858
Model:                            OLS   Adj. R-squared:                  0.851
Method:                 Least Squares   F-statistic:                     118.1
Date:                Fri, 04 Jun 2021   Prob (F-statistic):          4.17e-144
Time:                        20:32:01   Log-Likelihood:                 907.64
No. Observations:                 390   AIC:                            -1775.
Df Residuals:                     370   BIC:                            -1696.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.5069    

                            OLS Regression Results                            
Dep. Variable:                 perc_M   R-squared:                       0.865
Model:                            OLS   Adj. R-squared:                  0.858
Method:                 Least Squares   F-statistic:                     124.3
Date:                Fri, 04 Jun 2021   Prob (F-statistic):          1.35e-147
Time:                        20:32:01   Log-Likelihood:                 923.22
No. Observations:                 390   AIC:                            -1806.
Df Residuals:                     370   BIC:                            -1727.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.5061    

,shiftdays,alpha,Test Sample Size,Training Sample Size,Loglike Training,Loglike Test,RSS Training,RSS Test,NVar Training,NVar Test,...,Masks,MassGather50,MassGatherAll,Shops,Outdoor,Religion,Gatherings,Stayhome,Telework,Prikken
0,4,0.011548,44,389,915.011293,98.953275,0.206238,0.028680,1.545934,0.114485,...,-0.000068,0.018168,0.170541,0.122122,-0.054915,0.134692,-0.083465,-0.058351,0.154230,-0.008580
0,4,0.011548,44,389,908.571486,103.107994,0.213180,0.023744,1.521736,0.147421,...,0.006755,0.013771,0.176899,0.119100,-0.060082,0.131112,-0.075497,-0.055137,0.142071,-0.010103
0,4,0.011548,44,389,915.690062,95.709152,0.205519,0.033237,1.460151,0.206625,...,0.001283,-0.017278,0.086340,0.125789,-0.035902,0.068636,0.004527,-0.058296,-0.025596,-0.007455
0,4,0.011548,43,390,919.340656,93.732964,0.204682,0.032182,1.577324,0.086375,...,0.002799,0.017076,0.185134,0.128422,-0.063812,0.129470,-0.090069,-0.063652,0.150333,-0.003091
0,4,0.011548,43,390,907.636558,106.741171,0.217343,0.017573,1.535947,0.132587,...,0.001179,0.018146,0.177398,0.120999,-0.061020,0.120537,-0.079674,-0.053022,0.147012,-0.007121
0,4,0.011548,43,390,914.955625,97.733930,0.209337,0.026717,1.405052,0.257990,...,-0.002021,0.011856,0.158435,0.108935,-0.052588,0.103360,-0.069414,-0.060650,0.125248,0.001645
0,4,0.011548,43,390,917.464353,96.552846,0.206661,0.028226,1.481980,0.187588,...,0.002580,0.018199,0.180689,0.125010,-0.059656,0.133358,-0.086864,-0.062403,0.162430,-0.009406
0,4,0.011548,43,390,910.830927,103.409884,0.213812,0.020518,1.487978,0.180558,...,0.001593,0.013518,0.159207,0.116367,-0.053378,0.118558,-0.070557,-0.064087,0.130052,-0.004302
0,4,0.011548,43,390,923.217383,92.753222,0.200653,0.033682,1.481443,0.188185,...,0.000137,0.017048,0.177418,0.118504,-0.059436,0.121449,-0.084764,-0.060663,0.150878,-0.004553
0,4,0.011548,43,390,908.093943,106.682463,0.216834,0.017621,1.526943,0.142733,...,-0.000104,0.011812,0.158347,0.122039,-0.052950,0.127394,-0.074015,-0.060620,0.130151,-0.007439


In [71]:
DF.mean()

shiftdays                4.000000
alpha                    0.011548
Test Sample Size        43.300000
Training Sample Size   389.700000
Loglike Training       914.081229
Loglike Test            99.537690
RSS Training             0.209426
RSS Test                 0.026218
NVar Training            1.502449
NVar Test                0.164455
R2 Training              0.860490
R2 Test                  0.825234
F value                120.901066
P value                  0.000000
R2 Training2             0.860490
Intercept                0.506676
Workplace               -0.072409
Events                  -0.059045
Daycare                 -0.039041
Higher_Education        -0.008505
Secondary_Education      0.038640
Primary_Education        0.003616
Restaurants_Cafes        0.018684
Transport               -0.185292
Entertainment           -0.042990
Sports                  -0.175212
Indoor                  -0.132690
Masks                    0.001413
MassGather50             0.012231
MassGatherAll 

In [64]:
pd.options.display.float_format = "{:,.6f}".format
relatief = DF.fillna(0)#.divide(DF.Intercept, axis=0)
relatief = DF.quantile([0.5, .025, 0.975]).T
relatief.columns = ['median', 'low', 'high']
relatief.sort_values(by = ['median','high','low'], inplace = True)
relatiefGender = relatief.drop(['shiftdays', 'alpha', 'Test Sample Size', 'Training Sample Size', 'Loglike Training', 'Loglike Test', 
              'RSS Training', 'RSS Test', 'NVar Training', 'NVar Test', 'R2 Training', 'R2 Test', 'F value', 'P value'])
relatiefGender

,median,low,high
Transport,-0.190649,-0.197005,-0.144730
Sports,-0.174072,-0.191870,-0.167823
Indoor,-0.141112,-0.189969,-0.046435
Gatherings,-0.077357,-0.106776,-0.011342
Workplace,-0.069659,-0.094809,-0.060636
Events,-0.061603,-0.092699,-0.019179
Stayhome,-0.058866,-0.069577,-0.050792
Outdoor,-0.056716,-0.073980,-0.035904
Entertainment,-0.046010,-0.080904,0.005055
Daycare,-0.038764,-0.049888,-0.031552


# Age

In [41]:
%%time
shiftdays = range(0, 10) 
alphalist = np.power(10,np.linspace(-4,-1,num=17)) 
crossvalcount = 10

# shiftdays = [1, 2, 6]
# alphalist = [0.01, 0.5, 1e-3]


X = deep.drop(['No_cases', 'Rolling_R', 'perc_randstad', 'perc_under_30', 'perc_M'], axis=1)
Y = deep['perc_under_30']

curResDF = pd.DataFrame()
n = 0
totaln = len(shiftdays) * len(alphalist)
for s in shiftdays:
    
    X_used, Y_used = shift_outcome(X,Y,s,shift_date="y")
    
    X_used.columns = [col.replace(' ', '_').replace('/', '_') for col in X_used.columns.values]
    combined = X_used.copy()
    combined['perc_under_30'] = Y_used
    
    formula = 'perc_under_30 ~ ' + ' + '.join(X_used.columns)
    
    for a in alphalist:
        print(f'Huidige progressie: {(n / totaln)*100:.2f}% \n shiftday: {s}, alpha: {a}')
        kf = KFold(n_splits=crossvalcount, shuffle=True)

        for train_ind, test_ind in kf.split(X_used):
            #print(train_ind, test_ind)
            train = combined.iloc[train_ind]
            test = combined.iloc[test_ind]
#           
            trainmodel    = smf.ols(formula = formula, data = train)
            testmodel     = smf.ols(formula = formula, data = test)
        
            trainresults  = trainmodel.fit(alpha = a, L1_wt = 1)

            test_params   = []
            for paramname in testmodel.exog_names:
                if paramname in trainresults.params.keys():
                    test_params.append(trainresults.params[paramname])
                else:
                    test_params.append(0)

            obs_train     = np.array(trainmodel.endog)
            obs_test      = np.array(testmodel.endog)
            pred_train    = trainmodel.predict(trainresults.params)
            pred_test     = testmodel.predict(test_params)
            
            result_dict                          = {'shiftdays': s, 'alpha': a}
            result_dict['Test Sample Size']      = len(test)
            result_dict['Training Sample Size']  = len(train)
            result_dict['Loglike Training']      = trainmodel.loglike(trainresults.params)
            result_dict['Loglike Test']          = testmodel.loglike(np.array(test_params))
            result_dict['RSS Training']          = np.sum((obs_train - pred_train)**2)
            result_dict['RSS Test']              = np.sum((obs_test - pred_test)**2)
            result_dict['NVar Training']         = np.sum((obs_train - np.mean(obs_train))**2)
            result_dict['NVar Test']             = np.sum((obs_test - np.mean(obs_test))**2)
            result_dict['R2 Training']           = 1 - result_dict['RSS Training']/result_dict['NVar Training']
            result_dict['R2 Test']               = 1 - result_dict['RSS Test']/result_dict['NVar Test']
            result_dict['F value']               = trainresults.fvalue
            result_dict['P value']               = trainresults.f_pvalue

            result_dict.update({k:v for k,v in trainresults.params.items()})
            new_DF = pd.DataFrame({k:np.array([v]) for k,v in result_dict.items()})
            curResDF = curResDF.append(new_DF, ignore_index=False)
        n += 1

Huidige progressie: 0.00% 
 shiftday: 0, alpha: 0.0001
Huidige progressie: 0.59% 
 shiftday: 0, alpha: 0.0001539926526059492
Huidige progressie: 1.18% 
 shiftday: 0, alpha: 0.00023713737056616554
Huidige progressie: 1.76% 
 shiftday: 0, alpha: 0.0003651741272548377
Huidige progressie: 2.35% 
 shiftday: 0, alpha: 0.0005623413251903491
Huidige progressie: 2.94% 
 shiftday: 0, alpha: 0.0008659643233600654
Huidige progressie: 3.53% 
 shiftday: 0, alpha: 0.001333521432163324
Huidige progressie: 4.12% 
 shiftday: 0, alpha: 0.002053525026457146
Huidige progressie: 4.71% 
 shiftday: 0, alpha: 0.0031622776601683794
Huidige progressie: 5.29% 
 shiftday: 0, alpha: 0.004869675251658631
Huidige progressie: 5.88% 
 shiftday: 0, alpha: 0.007498942093324558
Huidige progressie: 6.47% 
 shiftday: 0, alpha: 0.011547819846894581
Huidige progressie: 7.06% 
 shiftday: 0, alpha: 0.01778279410038923
Huidige progressie: 7.65% 
 shiftday: 0, alpha: 0.027384196342643614
Huidige progressie: 8.24% 
 shiftday: 0, a

Huidige progressie: 71.18% 
 shiftday: 7, alpha: 0.00023713737056616554
Huidige progressie: 71.76% 
 shiftday: 7, alpha: 0.0003651741272548377
Huidige progressie: 72.35% 
 shiftday: 7, alpha: 0.0005623413251903491
Huidige progressie: 72.94% 
 shiftday: 7, alpha: 0.0008659643233600654
Huidige progressie: 73.53% 
 shiftday: 7, alpha: 0.001333521432163324
Huidige progressie: 74.12% 
 shiftday: 7, alpha: 0.002053525026457146
Huidige progressie: 74.71% 
 shiftday: 7, alpha: 0.0031622776601683794
Huidige progressie: 75.29% 
 shiftday: 7, alpha: 0.004869675251658631
Huidige progressie: 75.88% 
 shiftday: 7, alpha: 0.007498942093324558
Huidige progressie: 76.47% 
 shiftday: 7, alpha: 0.011547819846894581
Huidige progressie: 77.06% 
 shiftday: 7, alpha: 0.01778279410038923
Huidige progressie: 77.65% 
 shiftday: 7, alpha: 0.027384196342643614
Huidige progressie: 78.24% 
 shiftday: 7, alpha: 0.042169650342858224
Huidige progressie: 78.82% 
 shiftday: 7, alpha: 0.06493816315762113
Huidige progress

In [42]:
CVresults = curResDF.copy()
CVresults['alpha'] = CVresults['alpha'].map('{:.6e}'.format)
CVresults =  CVresults.groupby(['shiftdays','alpha'], as_index = False).agg(
            { 'Loglike Test':['mean','std'],
              'Loglike Training':['mean','std'],
              'R2 Test': ['mean','std'],
              'R2 Training': ['mean','std'],
              'RSS Training' : ['sum'],
              'RSS Test': ['sum'],
              'NVar Training':['sum'],
              'NVar Test':['sum'],
              'Test Sample Size':['sum'],
              'Training Sample Size':['sum']
            })
CVresults.columns = [ 'shiftdays','alpha',
                      'Loglike Test Avgd','Loglike Test Avgd Std',
                      'Loglike Training Avgd','Loglike Training Avgd Std',
                      'R2 Test Avgd','R2 Test Avgd Std',
                      'R2 Training Avgd','R2 Training Avgd Std',
                      'RSS Training Sum',
                      'RSS Test Sum',
                      'NVar Training Sum',
                      'NVar Test Sum',
                      'Test Sample Size',
                      'Training Sample Size'
                    ]
CVresults['RSS per datapoint Training'] = CVresults['RSS Training Sum']/CVresults['Training Sample Size']
CVresults['RSS per datapoint Test']     = CVresults['RSS Test Sum']/CVresults['Test Sample Size']

CVresults['R2 Training Weighted']       = 1 - CVresults['RSS Training Sum']/CVresults['NVar Training Sum']
CVresults['R2 Test Weighted']           = 1 - CVresults['RSS Test Sum']/CVresults['NVar Test Sum']

CVresults['alpha']                      = CVresults['alpha'].astype(np.float64) # return to numbers

# manually select best
CVresults.sort_values(by='R2 Test Weighted', ascending=False)

,shiftdays,alpha,Loglike Test Avgd,Loglike Test Avgd Std,Loglike Training Avgd,Loglike Training Avgd Std,R2 Test Avgd,R2 Test Avgd Std,R2 Training Avgd,R2 Training Avgd Std,RSS Training Sum,RSS Test Sum,NVar Training Sum,NVar Test Sum,Test Sample Size,Training Sample Size,RSS per datapoint Training,RSS per datapoint Test,R2 Training Weighted,R2 Test Weighted
17,1,0.100000,88.366660,6.692540,803.847529,7.505860,0.900333,0.047466,0.914708,0.003875,3.820755,0.459213,44.827083,4.950485,436,3924,0.000974,0.001053,0.914767,0.907239
14,0,0.064938,89.077561,8.150272,808.758087,9.241919,0.905132,0.038873,0.916376,0.004319,3.771938,0.462939,45.121812,4.977020,437,3933,0.000959,0.001059,0.916405,0.906985
13,0,0.000562,88.493189,6.285052,808.585223,7.355818,0.895591,0.055748,0.916257,0.004682,3.773445,0.460103,45.117967,4.941406,437,3933,0.000959,0.001053,0.916365,0.906888
15,0,0.007499,89.009887,8.258086,808.645002,7.856803,0.901893,0.037583,0.916351,0.002891,3.772849,0.460385,45.116911,4.932215,437,3933,0.000959,0.001054,0.916376,0.906658
2,0,0.011548,89.169082,8.302373,808.732285,9.290773,0.905751,0.036490,0.916382,0.003826,3.772450,0.461604,45.118170,4.943961,437,3933,0.000959,0.001056,0.916387,0.906633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,6,0.064938,83.401431,7.524281,773.855527,6.848768,0.875197,0.053626,0.901149,0.004207,4.204255,0.557610,42.546691,4.634226,431,3879,0.001084,0.001294,0.901185,0.879676
115,6,0.000562,83.652480,9.627927,773.940667,8.047866,0.875651,0.054586,0.901216,0.003635,4.202825,0.564043,42.550139,4.664851,431,3879,0.001083,0.001309,0.901227,0.879087
113,6,0.042170,82.872238,6.163937,774.100065,5.949670,0.870447,0.053619,0.901225,0.004547,4.198443,0.560365,42.544047,4.609760,431,3879,0.001082,0.001300,0.901315,0.878440
103,6,0.000100,83.892719,9.654654,774.112802,9.608334,0.862557,0.094783,0.901155,0.006227,4.200431,0.561318,42.543302,4.603180,431,3879,0.001083,0.001302,0.901267,0.878059


In [72]:
shiftdays = 1
alpha = 0.1
crossvalcount = 10

DF = pd.DataFrame()

X = deep.drop(['No_cases', 'Rolling_R', 'perc_randstad', 'perc_under_30', 'perc_M'], axis=1)
Y = deep['perc_under_30']

X_used, Y_used = shift_outcome(X,Y,shiftdays,shift_date="y")
X_used.columns = [col.replace(' ', '_').replace('/', '_') for col in X_used.columns.values]
combined = X_used.copy()
combined['perc_under_30'] = Y_used
formula = 'perc_under_30 ~ ' + ' + '.join(X_used.columns)
print(formula)
        
kf = KFold(n_splits=crossvalcount, shuffle=True)

for train_ind, test_ind in kf.split(X_used):
#   print(train_ind, test_ind)
    train = combined.iloc[train_ind]
    test = combined.iloc[test_ind]
       
    trainmodel    = smf.ols(formula = formula, data = train)
    testmodel     = smf.ols(formula = formula, data = test)

    trainresults  = trainmodel.fit(alpha = alpha, L1_wt = 1)#lasso

    test_params   = []
    for paramname in testmodel.exog_names:
        if paramname in trainresults.params.keys():
            test_params.append(trainresults.params[paramname])
        else:
            test_params.append(0)

    obs_train     = np.array(trainmodel.endog)
    obs_test      = np.array(testmodel.endog)
    pred_train    = trainmodel.predict(trainresults.params)
    pred_test     = testmodel.predict(test_params)

    result_dict                          = {'shiftdays': shiftdays, 'alpha': alpha}
    result_dict['Test Sample Size']      = len(test)
    result_dict['Training Sample Size']  = len(train)
    result_dict['Loglike Training']      = trainmodel.loglike(trainresults.params)
    result_dict['Loglike Test']          = testmodel.loglike(np.array(test_params))
    result_dict['RSS Training']          = np.sum((obs_train - pred_train)**2)
    result_dict['RSS Test']              = np.sum((obs_test - pred_test)**2)
    result_dict['NVar Training']         = np.sum((obs_train - np.mean(obs_train))**2)
    result_dict['NVar Test']             = np.sum((obs_test - np.mean(obs_test))**2)
    result_dict['R2 Training']           = 1 - result_dict['RSS Training']/result_dict['NVar Training']
    result_dict['R2 Test']               = 1 - result_dict['RSS Test']/result_dict['NVar Test']
    result_dict['F value']               = trainresults.fvalue
    result_dict['P value']               = trainresults.f_pvalue

    result_dict.update({k:v for k,v in trainresults.params.items()})
    new_DF = pd.DataFrame({k:np.array([v]) for k,v in result_dict.items()})
    DF = DF.append(new_DF, ignore_index=False)
    
# formula = 'Rolling_R ~ ' + ' + '.join(X_used.columns)  
# finalmodel = smf.ols(data = combined, formula = formula)
# finalresults = finalmodel.fit_regularized(alpha=alpha, L1_wt = 1) #lasso
DF

perc_under_30 ~ Workplace + Events + Daycare + Higher_Education + Secondary_Education + Primary_Education + Restaurants_Cafes + Transport + Entertainment + Sports + Indoor + Masks + MassGather50 + MassGatherAll + Shops + Outdoor + Religion + Gatherings + Stayhome + Telework + Prikken


,shiftdays,alpha,Test Sample Size,Training Sample Size,Loglike Training,Loglike Test,RSS Training,RSS Test,NVar Training,NVar Test,...,Masks,MassGather50,MassGatherAll,Shops,Outdoor,Religion,Gatherings,Stayhome,Telework,Prikken
0,1,0.100000,44,392,798.863282,92.692074,0.389673,0.038122,4.507143,0.464733,...,0.001865,-0.115829,-0.171826,0.063971,0.080610,0.234417,0.011309,-0.077658,0.050291,0.035124
0,1,0.100000,44,392,798.186210,93.167030,0.391021,0.037308,4.431605,0.549406,...,0.001792,-0.123811,-0.204993,0.062570,0.091205,0.226250,0.026182,-0.078229,0.026859,0.033126
0,1,0.100000,44,392,815.238861,78.452457,0.358439,0.072824,4.526564,0.419483,...,0.007230,-0.129739,-0.197632,0.059766,0.088725,0.219760,0.018320,-0.086583,0.023260,0.050232
0,1,0.100000,44,392,798.202724,93.795267,0.390988,0.036258,4.441810,0.536437,...,-0.003825,-0.117876,-0.175305,0.066574,0.083293,0.238680,0.009988,-0.085791,0.047949,0.037862
0,1,0.100000,44,392,803.409170,88.425801,0.380739,0.046280,4.588372,0.392793,...,0.003243,-0.127122,-0.208116,0.066445,0.093558,0.238362,0.026037,-0.089429,0.028715,0.039876
0,1,0.100000,44,392,795.506943,95.199609,0.396403,0.034015,4.560826,0.414009,...,0.005420,-0.124474,-0.209579,0.067648,0.100527,0.266683,0.019819,-0.098045,0.045317,0.035899
0,1,0.100000,43,393,797.948145,94.193009,0.396582,0.031501,4.400183,0.580242,...,0.004021,-0.120154,-0.187232,0.069755,0.086120,0.241184,0.019883,-0.082694,0.041061,0.037018
0,1,0.100000,43,393,813.446954,79.286554,0.366504,0.063012,4.468186,0.511050,...,0.008279,-0.118369,-0.181654,0.070650,0.086050,0.240438,0.003698,-0.090500,0.061440,0.037109
0,1,0.100000,43,393,800.944299,89.865261,0.390581,0.038525,4.533083,0.442011,...,0.009467,-0.125588,-0.189192,0.102303,0.084127,0.264990,0.021448,-0.075591,0.032582,0.039291
0,1,0.100000,43,393,821.764101,73.967900,0.351315,0.080698,4.365519,0.606524,...,0.005343,-0.113936,-0.161230,0.062419,0.078019,0.255438,-0.010181,-0.078097,0.090227,0.040673


In [73]:
DF.mean()

shiftdays                1.000000
alpha                    0.100000
Test Sample Size        43.600000
Training Sample Size   392.400000
Loglike Training       804.351069
Loglike Test            87.904496
RSS Training             0.381224
RSS Test                 0.047854
NVar Training            4.482329
NVar Test                0.491669
R2 Training              0.914941
R2 Test                  0.901109
F value                212.529357
P value                  0.000000
Intercept                0.082112
Workplace                0.117281
Events                   0.174658
Daycare                 -0.016352
Higher_Education         0.068552
Secondary_Education     -0.012112
Primary_Education       -0.058216
Restaurants_Cafes        0.015149
Transport               -0.344020
Entertainment            0.280669
Sports                  -0.103948
Indoor                   0.129021
Masks                    0.004284
MassGather50            -0.121690
MassGatherAll           -0.188676
Shops         

In [74]:
pd.options.display.float_format = "{:,.6f}".format
relatief = DF.fillna(0)#.divide(DF.Intercept, axis=0)
relatief = DF.quantile([0.5, .025, 0.975]).T
relatief.columns = ['median', 'low', 'high']
relatief.sort_values(by = ['median','high','low'], inplace = True)
relatiefAge = relatief.drop(['shiftdays', 'alpha', 'Test Sample Size', 'Training Sample Size', 'Loglike Training', 'Loglike Test', 
              'RSS Training', 'RSS Test', 'NVar Training', 'NVar Test', 'R2 Training', 'R2 Test', 'F value', 'P value'])
relatiefAge

,median,low,high
Transport,-0.342044,-0.367221,-0.328794
MassGatherAll,-0.188212,-0.209250,-0.163614
MassGather50,-0.121982,-0.129150,-0.114362
Sports,-0.104349,-0.109858,-0.093006
Stayhome,-0.084242,-0.096348,-0.076056
Primary_Education,-0.059481,-0.069250,-0.041632
Daycare,-0.013109,-0.030116,-0.009927
Secondary_Education,-0.010739,-0.037846,0.003662
Masks,0.004682,-0.002561,0.009200
Gatherings,0.019069,-0.007058,0.026149


# Location

In [45]:
%%time
shiftdays = range(0, 5) 
alphalist = np.power(10,np.linspace(-4,-1,num=17)) 
crossvalcount = 10

# shiftdays = [1, 2, 6]
# alphalist = [0.01, 0.5, 1e-3]


X = deep.drop(['No_cases', 'Rolling_R', 'perc_randstad', 'perc_under_30', 'perc_M'], axis=1)
Y = deep['perc_randstad']

curResDF = pd.DataFrame()
n = 0
totaln = len(shiftdays) * len(alphalist)
for s in shiftdays:
    
    X_used, Y_used = shift_outcome(X,Y,s,shift_date="y")
    
    X_used.columns = [col.replace(' ', '_').replace('/', '_') for col in X_used.columns.values]
    combined = X_used.copy()
    combined['perc_randstad'] = Y_used
    
    formula = 'perc_randstad ~ ' + ' + '.join(X_used.columns)
    
    for a in alphalist:
        print(f'Huidige progressie: {(n / totaln)*100:.2f}% \n shiftday: {s}, alpha: {a}')
        kf = KFold(n_splits=crossvalcount, shuffle=True)

        for train_ind, test_ind in kf.split(X_used):
            #print(train_ind, test_ind)
            train = combined.iloc[train_ind]
            test = combined.iloc[test_ind]
#           
            trainmodel    = smf.ols(formula = formula, data = train)
            testmodel     = smf.ols(formula = formula, data = test)
        
            trainresults  = trainmodel.fit(alpha = a, L1_wt = 1)

            test_params   = []
            for paramname in testmodel.exog_names:
                if paramname in trainresults.params.keys():
                    test_params.append(trainresults.params[paramname])
                else:
                    test_params.append(0)

            obs_train     = np.array(trainmodel.endog)
            obs_test      = np.array(testmodel.endog)
            pred_train    = trainmodel.predict(trainresults.params)
            pred_test     = testmodel.predict(test_params)
            
            result_dict                          = {'shiftdays': s, 'alpha': a}
            result_dict['Test Sample Size']      = len(test)
            result_dict['Training Sample Size']  = len(train)
            result_dict['Loglike Training']      = trainmodel.loglike(trainresults.params)
            result_dict['Loglike Test']          = testmodel.loglike(np.array(test_params))
            result_dict['RSS Training']          = np.sum((obs_train - pred_train)**2)
            result_dict['RSS Test']              = np.sum((obs_test - pred_test)**2)
            result_dict['NVar Training']         = np.sum((obs_train - np.mean(obs_train))**2)
            result_dict['NVar Test']             = np.sum((obs_test - np.mean(obs_test))**2)
            result_dict['R2 Training']           = 1 - result_dict['RSS Training']/result_dict['NVar Training']
            result_dict['R2 Test']               = 1 - result_dict['RSS Test']/result_dict['NVar Test']
            result_dict['F value']               = trainresults.fvalue
            result_dict['P value']               = trainresults.f_pvalue

            result_dict.update({k:v for k,v in trainresults.params.items()})
            new_DF = pd.DataFrame({k:np.array([v]) for k,v in result_dict.items()})
            curResDF = curResDF.append(new_DF, ignore_index=False)
        n += 1

Huidige progressie: 0.00% 
 shiftday: 0, alpha: 0.0001
Huidige progressie: 1.18% 
 shiftday: 0, alpha: 0.0001539926526059492
Huidige progressie: 2.35% 
 shiftday: 0, alpha: 0.00023713737056616554
Huidige progressie: 3.53% 
 shiftday: 0, alpha: 0.0003651741272548377
Huidige progressie: 4.71% 
 shiftday: 0, alpha: 0.0005623413251903491
Huidige progressie: 5.88% 
 shiftday: 0, alpha: 0.0008659643233600654
Huidige progressie: 7.06% 
 shiftday: 0, alpha: 0.001333521432163324
Huidige progressie: 8.24% 
 shiftday: 0, alpha: 0.002053525026457146
Huidige progressie: 9.41% 
 shiftday: 0, alpha: 0.0031622776601683794
Huidige progressie: 10.59% 
 shiftday: 0, alpha: 0.004869675251658631
Huidige progressie: 11.76% 
 shiftday: 0, alpha: 0.007498942093324558
Huidige progressie: 12.94% 
 shiftday: 0, alpha: 0.011547819846894581
Huidige progressie: 14.12% 
 shiftday: 0, alpha: 0.01778279410038923
Huidige progressie: 15.29% 
 shiftday: 0, alpha: 0.027384196342643614
Huidige progressie: 16.47% 
 shiftday

In [46]:
CVresults = curResDF.copy()
CVresults['alpha'] = CVresults['alpha'].map('{:.6e}'.format)
CVresults =  CVresults.groupby(['shiftdays','alpha'], as_index = False).agg(
            { 'Loglike Test':['mean','std'],
              'Loglike Training':['mean','std'],
              'R2 Test': ['mean','std'],
              'R2 Training': ['mean','std'],
              'RSS Training' : ['sum'],
              'RSS Test': ['sum'],
              'NVar Training':['sum'],
              'NVar Test':['sum'],
              'Test Sample Size':['sum'],
              'Training Sample Size':['sum']
            })
CVresults.columns = [ 'shiftdays','alpha',
                      'Loglike Test Avgd','Loglike Test Avgd Std',
                      'Loglike Training Avgd','Loglike Training Avgd Std',
                      'R2 Test Avgd','R2 Test Avgd Std',
                      'R2 Training Avgd','R2 Training Avgd Std',
                      'RSS Training Sum',
                      'RSS Test Sum',
                      'NVar Training Sum',
                      'NVar Test Sum',
                      'Test Sample Size',
                      'Training Sample Size'
                    ]
CVresults['RSS per datapoint Training'] = CVresults['RSS Training Sum']/CVresults['Training Sample Size']
CVresults['RSS per datapoint Test']     = CVresults['RSS Test Sum']/CVresults['Test Sample Size']

CVresults['R2 Training Weighted']       = 1 - CVresults['RSS Training Sum']/CVresults['NVar Training Sum']
CVresults['R2 Test Weighted']           = 1 - CVresults['RSS Test Sum']/CVresults['NVar Test Sum']

CVresults['alpha']                      = CVresults['alpha'].astype(np.float64) # return to numbers

# manually select best
CVresults.sort_values(by='R2 Test Weighted', ascending=False)

,shiftdays,alpha,Loglike Test Avgd,Loglike Test Avgd Std,Loglike Training Avgd,Loglike Training Avgd Std,R2 Test Avgd,R2 Test Avgd Std,R2 Training Avgd,R2 Training Avgd Std,RSS Training Sum,RSS Test Sum,NVar Training Sum,NVar Test Sum,Test Sample Size,Training Sample Size,RSS per datapoint Training,RSS per datapoint Test,R2 Training Weighted,R2 Test Weighted
4,0,0.000154,78.789093,4.548787,724.620970,5.432492,0.869854,0.028851,0.883430,0.003222,5.782007,0.707323,49.607324,5.481508,437,3933,0.001470,0.001619,0.883444,0.870962
52,3,0.000100,78.498474,5.225479,718.385126,5.426958,0.870053,0.024704,0.881760,0.002629,5.779661,0.699294,48.881412,5.400623,434,3906,0.001480,0.001611,0.881762,0.870516
62,3,0.042170,78.503710,5.420389,718.440106,5.891267,0.862553,0.050916,0.881715,0.004278,5.778369,0.700781,48.873685,5.331074,434,3906,0.001479,0.001615,0.881769,0.868548
64,3,0.000562,78.555884,5.588834,718.505901,6.002398,0.867239,0.029071,0.881792,0.003066,5.777015,0.704392,48.876183,5.353027,434,3906,0.001479,0.001623,0.881803,0.868412
3,0,0.001334,79.699317,8.334207,724.751816,8.331510,0.867811,0.038075,0.883445,0.004003,5.781098,0.709596,49.597523,5.392261,437,3933,0.001470,0.001624,0.883440,0.868405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,4,0.027384,76.659706,5.687118,708.218159,5.945843,0.854724,0.023386,0.875365,0.002183,6.024159,0.756429,48.330297,5.198529,433,3897,0.001546,0.001747,0.875354,0.854492
83,4,0.007499,76.978830,5.286341,708.101581,6.034785,0.848654,0.048523,0.875165,0.005158,6.027988,0.744020,48.319587,5.103314,433,3897,0.001547,0.001718,0.875248,0.854209
79,4,0.042170,76.853343,6.958827,708.487112,7.938402,0.853956,0.030076,0.875513,0.002649,6.017894,0.762798,48.331611,5.210016,433,3897,0.001544,0.001762,0.875487,0.853590
75,4,0.000237,77.381040,9.207132,708.519029,10.082503,0.852068,0.048477,0.875462,0.005022,6.018991,0.763778,48.325134,5.151751,433,3897,0.001545,0.001764,0.875448,0.851744


In [75]:
shiftdays = 0
alpha = 0.000154
crossvalcount = 10

DF = pd.DataFrame()

X = deep.drop(['No_cases', 'Rolling_R', 'perc_randstad', 'perc_under_30', 'perc_M'], axis=1)
Y = deep['perc_randstad']

X_used, Y_used = shift_outcome(X,Y,shiftdays,shift_date="y")
X_used.columns = [col.replace(' ', '_').replace('/', '_') for col in X_used.columns.values]
combined = X_used.copy()
combined['perc_randstad'] = Y_used
formula = 'perc_randstad ~ ' + ' + '.join(X_used.columns)
print(formula)
        
kf = KFold(n_splits=crossvalcount, shuffle=True)

for train_ind, test_ind in kf.split(X_used):
#   print(train_ind, test_ind)
    train = combined.iloc[train_ind]
    test = combined.iloc[test_ind]
       
    trainmodel    = smf.ols(formula = formula, data = train)
    testmodel     = smf.ols(formula = formula, data = test)

    trainresults  = trainmodel.fit(alpha = alpha, L1_wt = 1)#lasso

    test_params   = []
    for paramname in testmodel.exog_names:
        if paramname in trainresults.params.keys():
            test_params.append(trainresults.params[paramname])
        else:
            test_params.append(0)

    obs_train     = np.array(trainmodel.endog)
    obs_test      = np.array(testmodel.endog)
    pred_train    = trainmodel.predict(trainresults.params)
    pred_test     = testmodel.predict(test_params)

    result_dict                          = {'shiftdays': shiftdays, 'alpha': alpha}
    result_dict['Test Sample Size']      = len(test)
    result_dict['Training Sample Size']  = len(train)
    result_dict['Loglike Training']      = trainmodel.loglike(trainresults.params)
    result_dict['Loglike Test']          = testmodel.loglike(np.array(test_params))
    result_dict['RSS Training']          = np.sum((obs_train - pred_train)**2)
    result_dict['RSS Test']              = np.sum((obs_test - pred_test)**2)
    result_dict['NVar Training']         = np.sum((obs_train - np.mean(obs_train))**2)
    result_dict['NVar Test']             = np.sum((obs_test - np.mean(obs_test))**2)
    result_dict['R2 Training']           = 1 - result_dict['RSS Training']/result_dict['NVar Training']
    result_dict['R2 Test']               = 1 - result_dict['RSS Test']/result_dict['NVar Test']
    result_dict['F value']               = trainresults.fvalue
    result_dict['P value']               = trainresults.f_pvalue

    result_dict.update({k:v for k,v in trainresults.params.items()})
    new_DF = pd.DataFrame({k:np.array([v]) for k,v in result_dict.items()})
    DF = DF.append(new_DF, ignore_index=False)
    
# formula = 'Rolling_R ~ ' + ' + '.join(X_used.columns)  
# finalmodel = smf.ols(data = combined, formula = formula)
# finalresults = finalmodel.fit_regularized(alpha=alpha, L1_wt = 1) #lasso
DF

perc_randstad ~ Workplace + Events + Daycare + Higher_Education + Secondary_Education + Primary_Education + Restaurants_Cafes + Transport + Entertainment + Sports + Indoor + Masks + MassGather50 + MassGatherAll + Shops + Outdoor + Religion + Gatherings + Stayhome + Telework + Prikken


,shiftdays,alpha,Test Sample Size,Training Sample Size,Loglike Training,Loglike Test,RSS Training,RSS Test,NVar Training,NVar Test,...,Masks,MassGather50,MassGatherAll,Shops,Outdoor,Religion,Gatherings,Stayhome,Telework,Prikken
0,0,0.000154,44,393,732.081839,69.982513,0.554508,0.107023,4.971621,0.530667,...,0.084993,-0.127897,-0.058086,0.182228,-0.003967,0.306516,-0.094669,0.067365,0.078503,-0.093039
0,0,0.000154,44,393,717.403389,87.313894,0.597515,0.048679,4.988104,0.506319,...,0.061957,-0.110838,-0.048464,0.101469,0.000121,0.231081,-0.101259,0.055448,0.091318,-0.094608
0,0,0.000154,44,393,739.327422,66.438428,0.534433,0.125730,5.022108,0.453117,...,0.062487,-0.107336,-0.083460,0.086737,0.016401,0.250502,-0.079634,0.027399,0.078703,-0.120113
0,0,0.000154,44,393,722.155808,80.872239,0.583238,0.065239,4.840409,0.670491,...,0.055770,-0.109986,-0.068887,0.095037,0.007949,0.226205,-0.088089,0.042520,0.076075,-0.092056
0,0,0.000154,44,393,718.570321,85.231575,0.593977,0.053512,4.883505,0.625138,...,0.059639,-0.124733,-0.100291,0.087573,0.014808,0.200051,-0.056241,0.038219,0.019349,-0.092379
0,0,0.000154,44,393,723.249818,79.294666,0.579999,0.070089,4.847562,0.663802,...,0.053097,-0.109677,-0.058479,0.097457,0.003981,0.235247,-0.089595,0.041583,0.093323,-0.099860
0,0,0.000154,44,393,714.491253,91.241845,0.606436,0.040720,4.929608,0.582587,...,0.057765,-0.108393,-0.050603,0.098018,-0.000408,0.226105,-0.102178,0.051605,0.084964,-0.092375
0,0,0.000154,43,394,736.033194,69.449283,0.550056,0.099572,4.969251,0.542902,...,0.055865,-0.106716,-0.064714,0.089758,0.008351,0.223378,-0.104613,0.056565,0.096477,-0.100696
0,0,0.000154,43,394,716.470337,88.848339,0.607483,0.040391,5.064446,0.425126,...,0.050573,-0.105319,-0.057830,0.099027,0.004452,0.235749,-0.096099,0.052235,0.094780,-0.098964
0,0,0.000154,43,394,732.935608,70.751886,0.558773,0.093718,5.084711,0.427589,...,0.056701,-0.121918,-0.051730,0.059863,-0.000404,0.176383,-0.090050,0.059617,0.079948,-0.092398


In [76]:
DF.mean()

shiftdays                0.000000
alpha                    0.000154
Test Sample Size        43.700000
Training Sample Size   393.300000
Loglike Training       725.271899
Loglike Test            78.942467
RSS Training             0.576642
RSS Test                 0.074467
NVar Training            4.960132
NVar Test                0.542774
R2 Training              0.883694
R2 Test                  0.856873
F value                150.522699
P value                  0.000000
Intercept                0.364019
Workplace                0.294911
Events                  -0.017939
Daycare                  0.050620
Higher_Education         0.018288
Secondary_Education     -0.021320
Primary_Education       -0.109617
Restaurants_Cafes       -0.027496
Transport               -0.043779
Entertainment            0.034256
Sports                  -0.153191
Indoor                   0.041439
Masks                    0.059885
MassGather50            -0.113281
MassGatherAll           -0.064254
Shops         

In [57]:
pd.options.display.float_format = "{:,.6f}".format
relatief = DF.fillna(0)#.divide(DF.Intercept, axis=0)
relatief = DF.quantile([0.5, .025, 0.975]).T
relatief.columns = ['median', 'low', 'high']
relatief.sort_values(by = ['median','high','low'], inplace = True)
relatiefLOC = relatief.drop(['shiftdays', 'alpha', 'Test Sample Size', 'Training Sample Size', 'Loglike Training', 'Loglike Test', 
              'RSS Training', 'RSS Test', 'NVar Training', 'NVar Test', 'R2 Training', 'R2 Test', 'F value', 'P value'])
relatiefLOC

,median,low,high
Sports,-0.154752,-0.166327,-0.137094
MassGather50,-0.110953,-0.125064,-0.104888
Primary_Education,-0.106657,-0.120742,-0.098200
Prikken,-0.095983,-0.106156,-0.089135
Gatherings,-0.090030,-0.110905,-0.063058
MassGatherAll,-0.065459,-0.099797,-0.031528
Transport,-0.037256,-0.061082,-0.031368
Restaurants_Cafes,-0.030398,-0.045769,0.008285
Events,-0.019769,-0.039453,0.006138
Secondary_Education,-0.018577,-0.070524,-0.002202


In [77]:
display(relatiefGender, relatiefAge, relatiefLOC)

,median,low,high
Transport,-0.190649,-0.197005,-0.144730
Sports,-0.174072,-0.191870,-0.167823
Indoor,-0.141112,-0.189969,-0.046435
Gatherings,-0.077357,-0.106776,-0.011342
Workplace,-0.069659,-0.094809,-0.060636
Events,-0.061603,-0.092699,-0.019179
Stayhome,-0.058866,-0.069577,-0.050792
Outdoor,-0.056716,-0.073980,-0.035904
Entertainment,-0.046010,-0.080904,0.005055
Daycare,-0.038764,-0.049888,-0.031552


,median,low,high
Transport,-0.342044,-0.367221,-0.328794
MassGatherAll,-0.188212,-0.209250,-0.163614
MassGather50,-0.121982,-0.129150,-0.114362
Sports,-0.104349,-0.109858,-0.093006
Stayhome,-0.084242,-0.096348,-0.076056
Primary_Education,-0.059481,-0.069250,-0.041632
Daycare,-0.013109,-0.030116,-0.009927
Secondary_Education,-0.010739,-0.037846,0.003662
Masks,0.004682,-0.002561,0.009200
Gatherings,0.019069,-0.007058,0.026149


,median,low,high
Sports,-0.154752,-0.166327,-0.137094
MassGather50,-0.110953,-0.125064,-0.104888
Primary_Education,-0.106657,-0.120742,-0.098200
Prikken,-0.095983,-0.106156,-0.089135
Gatherings,-0.090030,-0.110905,-0.063058
MassGatherAll,-0.065459,-0.099797,-0.031528
Transport,-0.037256,-0.061082,-0.031368
Restaurants_Cafes,-0.030398,-0.045769,0.008285
Events,-0.019769,-0.039453,0.006138
Secondary_Education,-0.018577,-0.070524,-0.002202


In [79]:
relatiefGender.reset_index().rename({'index':'Maatregelgroep'}, axis=1).to_csv('Resultaten/LASSO gender.csv', sep=';', index=False)
relatiefAge.reset_index().rename({'index':'Maatregelgroep'}, axis=1).to_csv('Resultaten/LASSO age.csv', sep=';', index=False)
relatiefLOC.reset_index().rename({'index':'Maatregelgroep'}, axis=1).to_csv('Resultaten/LASSO location.csv', sep=';', index=False)







